In [ ]:
# https://answers.opencv.org/question/97416/replace-a-range-of-colors-with-a-specific-color-in-python/

In [66]:
import numpy as np
import cv2 as cv

img = cv.imread('data_hierarchy2.png')
img_white_bg = cv.imread('data_hierarchy3.png')

"""缩小图像,方便看效果
""" 
def resizeImg(src):
    height, width = src.shape[:2]
    size = (int(width * 0.3), int(height * 0.3))  # bgr
    img = cv.resize(src, size, interpolation=cv.INTER_AREA)
    return img

"""找出ROI，用于分割原图
原图有三块区域，一个是地块区域，一个是颜色示例区域，一个距离标尺区域
假设倒排后的轮廓列表是按照0地块，1标尺，2...N颜色排列
"""
def findROIContours(src):
    copy = src.copy()
    gray = cv.cvtColor(copy, cv.COLOR_BGR2GRAY)
    # cv.imshow("gray", gray)
    
    # 低于thresh都变为黑色，maxval是给binary用的
    # 白底 254, 255 黑底 0, 255
    threshold = cv.threshold(gray, 0, 255, cv.THRESH_BINARY)[1]
    # cv.imshow("threshold", threshold)
    contours, hierarchy = cv.findContours(threshold, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    sortedCnts = sorted(contours, key = cv.contourArea, reverse=True)
    # cv.drawContours(copy, [maxCnt], -1, (255, 0, 0), 2)
    # cv.imshow("roi contours", copy)
    return sortedCnts
    


"""按照mask，截取roi
"""
def getROIByContour(src, cnt):
    copy = src.copy()
    mask = np.zeros(copy.shape[:2], np.uint8)
    mask = cv.fillConvexPoly(mask, cnt, (255,255,255)) 
    # cv.imshow("mask", mask)
    # print(mask.shape)
    # print(copy.dtype)
    roi = cv.bitwise_and(copy, copy, mask=mask)
    # cv.imshow("roi", roi)
    return roi

"""找出所有的地块轮廓
"""
def findAllBlockContours(src):
    copy = src.copy()
    cv.imshow("findAllBlockContours copy", copy)
    gray = cv.cvtColor(copy, cv.COLOR_BGR2GRAY)
    cv.imshow("findAllBlockContours gray", gray)
    # 这个canny的threshold到底该怎么计算？
    edges = cv.Canny(gray, 60, 150)
    cv.imshow("findAllBlockContours edges", edges)
    contours, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    for cnt in contours:
        if cv.contourArea(cnt) > 1000:
            cv.drawContours(copy, [cnt], -1, (255, 0, 0), 2)
            
    cv.imshow("findAllBlockContours", copy)
    return contours

"""找出颜色示例里的颜色BGR
"""
def findBGRColors(cnts):
    colors = []
    for cnt in cnts:
        epsilon = 0.01 * cv.arcLength(cnt, True)
        approxCurve = cv.approxPolyDP(cnt, epsilon, True)
        
        if len(approxCurve) == 4 and cv.contourArea(cnt) > 1000:
            x,y,w,h = cv.boundingRect(cnt)
            # 缩小矩形,去掉边上的干扰
            x1,y1,x2,y2 = x+5, y+5, x+w-5, y+h-5
            rect = np.array([[x1,y1],[x2,y1],[x2,y2],[x1,y2]])
            # print(rect)
            colorRegion = getROIByContour(img_white_bg, rect)
            meanMask = np.zeros(img_white_bg.shape[:2], np.uint8)
            meanMask[y1:y2, x1:x2]=255
            # cv.imshow("findColorRegions meanMask", meanMask)
            b,g,r,a = cv.mean(colorRegion, meanMask)
            # print(r,g,b)
            # cv.imshow("findColorRegions", colorRegion)
            colors.append((b,g,r))
    return colors





img = resizeImg(img)
img_white_bg = resizeImg(img_white_bg)
sortedCnts = findROIContours(img)    
rootRegion = getROIByContour(img_white_bg, sortedCnts[0])
# blockCnts = findAllBlockContours(rootRegion)
# print(len(sortedCnts[2:]))
bgrColors = findBGRColors(sortedCnts[2:])
print(bgrColors)

# # # BGR转化为HSV
# HSV = cv.cvtColor(img, cv.COLOR_BGR2HSV)
# cv.imshow("imageHSV", HSV)
# cv.imshow('image', img)

for bgr in bgrColors:
    b,g,r = bgr[0],bgr[1],bgr[2]
    lower = np.array([b,g,r], np.uint8)
    upper = np.array([b+5,g+5,r+5], np.uint8)
    print(upper)
    mask = cv.inRange(img_white_bg, lower, upper)
    cv.imshow("images", np.hstack([img_white_bg, mask]))
#     img_white_bg[np.where((img_white_bg == [0,0,0]).all(axis = 2))] = bgr
#     mask[np.where(black_mask == [0])] = [255]

    cv.imshow("mask", mask)
    cv.waitKey(0)
    
# color = hsvLowAndUpList
# for (lower, upper) in color:
#     lower = np.array(lower, dtype="uint8")  
#     upper = np.array(upper, dtype="uint8")  
#     # 根据阈值找到对应颜色区域
#     mask = cv.inRange(HSV, lower, upper) 
#     mask = 255 - mask 
#     output = cv.bitwise_and(img, img, mask=mask) 

#     # output = cv.cvtColor(output,cv.COLOR_HSV2BGR)

#     # 展示图片
#     cv.imshow("images", np.hstack([img, output]))
#     contours, hierarchy = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
#     print(mask.shape)
#     # print(mask[0])
#     print(len(contours))
#     cv.drawContours(img, contours, -1, (0, 0, 255), 1)
#     for i in contours:
#         print(cv.contourArea(i))  # 计算缺陷区域面积
#         x, y, w, h = cv.boundingRect(i)  # 画矩形框
#         cv.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 1)
#     # cv.imwrite(show_result_path, match_img_color)
#     cv.imshow("detect", img)
#     cv.imshow("chanle", img)
#     cv.waitKey(0)

cv.waitKey(0)
cv.destroyAllWindows()


[(250.0, 0.0, 250.0), (0.0, 250.0, 250.0), (39.0, 103.0, 201.0), (250.0, 250.0, 0.0), (0.0, 250.0, 0.0), (9.0, 9.0, 250.0)]
[255   5 255]


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)

In [63]:
import numpy as np
import cv2 as cv


"""缩小图像,方便看效果
""" 
def resizeImg(src):
    height, width = src.shape[:2]
    size = (int(width * 0.3), int(height * 0.3))  # bgr
    img = cv.resize(src, size, interpolation=cv.INTER_AREA)
    return img

img = cv.imread('data_hierarchy2.png')
img_white_bg = cv.imread('data_hierarchy3.png')
img = resizeImg(img)
img_white_bg = resizeImg(img_white_bg)

img_white_bg_ll = cv.Laplacian(img_white_bg, -1, 3)
cv.imshow("images", np.hstack((img_white_bg, img_white_bg_ll)))

binary = cv.threshold(img_white_bg_ll, 0, 255, cv.THRESH_BINARY_INV)[1]
cv.imshow("binary", binary)

# edges = cv.Canny(img_white_bg_ll, 60,150)
# contours, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
# copy = img_white_bg.copy()
# cv.drawContours(copy, contours, -1, (0, 0, 255), 1)
# cv.imshow("copy", copy)

cv.waitKey(0)
cv.destroyAllWindows()